In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np

# Загрузка CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Определение модели Mixture of Experts (MoE)
class MoE(nn.Module):
    def __init__(self, input_dim, num_classes, num_experts=4):
        super(MoE, self).__init__()
        self.num_experts = num_experts

        # Определяем экспертов
        self.experts = nn.ModuleList([nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        ) for _ in range(num_experts)])

        # Гейт для выбора экспертов
        self.gate = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        # Применяем гейт к входным данным
        batch_size = x.size(0)
        gate_outputs = self.gate(x.view(batch_size, -1))  # Приводим к вектору
        gate_weights = torch.softmax(gate_outputs, dim=-1)  # Получаем вероятности выбора экспертов

        # Получаем выходы от каждого эксперта
        expert_outputs = [expert(x) for expert in self.experts]

        # Смешиваем выходы экспертов по весам
        output = sum(gate_weights[:, i].view(batch_size, 1) * expert_outputs[i] for i in range(self.num_experts))

        return output

# Обучение модели
def train_model(model, train_loader, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Тестирование модели
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

# Инициализация и обучение модели
input_dim = 3 * 32 * 32  # Размер входного изображения CIFAR-10
num_classes = 10          # Количество классов CIFAR-10

model = MoE(input_dim=input_dim, num_classes=num_classes)
train_model(model, train_loader)
test_model(model, test_loader)


Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 

In [14]:
test_model(model, test_loader)

Accuracy of the model on the test images: 55.27%
